In [14]:
%conda install -c anaconda libev
%conda install -c msys2 m2-make
%conda install -c conda-forge pkg-config
%pip install cassandra-driver
%pip install psycopg2

Channels:
 - anaconda
 - defaults
Platform: win-64
Solving environment: failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - libev

Current channels:

  - https://conda.anaconda.org/anaconda
  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




Channels:
 - msys2
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
    current version: 25.3.1
    latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random
import uuid
from datetime import datetime, timedelta
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'])
session = cluster.connect('my_keyspace')

def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds()))
    )

def create() -> None:
    query = """
    create table if not exists temperature_day (
        sensor_id text,
        date text,
        event_time timestamp,
        temperature float,
        primary key((sensor_id, date), event_time)
    )
    """
    session.execute(query)
    print("Tabla temperature_day creada correctamente.")

def drop() -> None:
    query = "drop table if exists temperature_day"
    session.execute(query)
    print("Tabla temperature_day eliminada correctamente.")

def insert() -> None:
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2024, 1, 1)

    n = 100

    for _ in range(n):
        sensor_id = str(uuid.uuid4())
        date = random_date(start_date, end_date).strftime('%Y-%m-%d')
        event_time = random_date(start_date, end_date)
        temperature = random.uniform(-30.0, 50.0)

        query = """
        insert into temperature_day(sensor_id, date, event_time, temperature)
        values (%s, %s, %s, %s)
        """

        session.execute(query, (sensor_id, date, event_time, temperature))

    print(f"{n} filas insertadas correctamente en la tabla temperature_day.")

drop()
create()
insert()